In [1]:
from imutils.video import FileVideoStream
from imutils import face_utils
import sys
import datetime
import argparse
import math, operator
import imutils
import time
import dlib
import cv2
import numpy as np
from threading import Thread
from queue import Queue
# from main import get_dict
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from PIL import Image
import face_recognition

In [2]:
dataScript = pd.read_csv('../script/script_csv.csv')
dataScript['starttime'] = dataScript['starttime'].str.replace(',','.')
dataScript['endtime'] = dataScript['endtime'].str.replace(',','.')



In [3]:
detector = dlib.get_frontal_face_detector()
shape_predictor='facial-landmarks/shape_predictor.dat'
predictor = dlib.shape_predictor(shape_predictor)

In [4]:
fvs = FileVideoStream('friendss06e05.mkv').start()
time.sleep(1.0)
frame_count = 0
df_pos = 0

In [5]:
prev_diff_array = None
flag =0
fp = open('temp.txt','a')
print("Total frame count:", fvs.stream.get(cv2.CAP_PROP_FRAME_COUNT))
fps = round(fvs.stream.get(cv2.CAP_PROP_FPS))
print("FPS:", fps)

Total frame count: 31561.0
FPS: 24


In [6]:
f=open("temp_frame.txt","w+",buffering=1)

In [7]:
list_train=[]
cnt=1
while fvs.more():
    frame = fvs.read()
    curr_time = datetime.datetime.utcfromtimestamp(frame_count/fps)
    list_time=str(curr_time).split(" ")[1]
    frame = imutils.resize(frame, width=500)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 1)
    diff_array = 0
    for (i, rect) in enumerate(rects):
        if(len(rects)==1):
            shape = predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)
            (x, y, w, h) = face_utils.rect_to_bb(rect)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cropped_image = frame[y:y+h, x:x+w]
            if(cropped_image.shape[0]>=20 and cropped_image.shape[1]>=20):
                f.write(str(cropped_image.shape)+'\n')
                cropped_image = cv2.resize(cropped_image, (64,64), fx=0.5, fy=0.5, interpolation = cv2.INTER_AREA)
                cv2.imwrite('sample_images/crop'+str(cnt)+'.jpg', cropped_image)
                encodings = face_recognition.face_encodings(cropped_image)
                s=dataScript[(dataScript['endtime'] >= list_time) & (dataScript['starttime'] <= list_time)]
                list_s=s.values.tolist()
                if(len(encodings)!=0 and len(list_s)!=0):
                    encode = encodings[0]
                    output = [str(x) for x in encode]
                    output.append(list_s[0][2])
                    list_train.append(output)
#                     print(output)
#                     encodings.append(list_s[0][2])
#                     encodings.append(s['name'])
#                     print(encodings)
#                     train_data = train_data.append(pd.DataFrame({'feature': encodings, 'Name': s['name']}, index=[0]), ignore_index=True)
                cnt+=1
            
            
    frame_count += 1
    cv2.imshow("Frame ", frame)
    cv2.waitKey(1)

cv2.destroyAllWindows()
fvs.stop()

AttributeError: 'NoneType' object has no attribute 'shape'

In [8]:
col_names=[]

for i in range(1,129):
    col_names.append('f'+str(i))
col_names.append('Name')

train_data = pd.DataFrame(list_train,columns=col_names)
train_data.to_csv('feature_data.csv',index=False)

In [9]:
train_data.shape

(10432, 129)